In [1]:
import pandas as pd
import json
import datetime
import requests

### List of cities and Ticketmaster codes
City_List = 
    'Denver' : 264,
    'San Francisco Bay': 382,
    'Portland': 362,
    'Los Angeles': 27,
    'Las Vegas' : 14,
    'Phoenix': 36, 
    'Seattle': 42,
    'Austin' : 40, 
    'Houston': 22,
    'Dallas' : 5,
    'Chicago': 3,
    'Nashville': 31,
    'Atlanta': 10,
    'Boston' : 11,
    'New York': 35,
    'Washington DC': 47,
    'Miami' : 15
### Ticketmaster API info    
base_url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US&apikey={apikey}'
api_key = 'OhKdHqBZOOuGCrWIcjlhzoxmnjUoaGWL'
dmaId = [382,362,264]
marketId =  [42,27,14,36,40,22,5,3,31,10,11,35,47,15]



### Define Functions

Define functions for pulling event data from Ticketmaster API and formatting dataframe

In [2]:
#DMA and Market numbers
dma_list = [382,362,264]
mkt_list =  [42,27,14,36,40,22,5,3,31,10,11,35,47,15]

# Function to get the number of pages of ticketmaster data
def get_number_of_TM_pages(dma=None,market=None):
    url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'
    payload = {'dmaId': dma, 
               'marketId': market, 
               'source': 'ticketmaster,frontgate',
               'classificationName': 'music',
               'size': '200',
               'apikey':'OhKdHqBZOOuGCrWIcjlhzoxmnjUoaGWL'}
    r = requests.get(url,params=payload,verify=True)
    json_obj = json.loads(r.text)
    return json_obj['page']['totalPages']

#Get TicketMaster data, return a dataframe
def getTicketMasterData(dma=None,market=None,page=None):
    url = 'https://app.ticketmaster.com/discovery/v2/events.json?countryCode=US'
    payload = {'dmaId': dma, 
               'marketId': market, 
               'source': 'ticketmaster,frontgate',
               'classificationName' : 'music',
               'sort': 'date,name,asc',
               'size': '200',
               'page': page,
               'apikey':'OhKdHqBZOOuGCrWIcjlhzoxmnjUoaGWL'}
    r = requests.get(url,params=payload,verify=True)
    json_response = json.loads(r.text)
    event_info = []
    for event in json_response.get('_embedded',{}).get('events',{}):
         event_info.append({
            'id': event.get('id',{}),
            'name' : event.get('name',{}),
            'artist': list(attraction.get('name') for attraction in event['_embedded'].get('attractions',{})),
            'venue' : list(venue.get('name') for venue in event['_embedded'].get('venues',{})),
            'venue_city' : list(venue.get('city',{}).get('name') for venue in event['_embedded'].get('venues',{})),
            'venue_state' : list(venue.get('state',{}).get('name') for venue in event['_embedded'].get('venues',{})),
            'description' : event.get('description',{}),
            'more_info' : event.get('additionalInfo',{}),
            'start_date' : event.get('dates',{}).get('start',{}).get('dateTime',{}),
            'timezone' : event.get('dates',{}).get('timezone'),
            'span_multiple_days' : event.get('dates',{}).get('spanMultipleDays'),
            'presale_date_start' : list(presale.get('startDateTime') for presale in event.get('sales').get('presales',{})),
            'presale_date_end' : list(presale.get('endDateTime') for presale in event.get('sales').get('presales',{})),
            'sale_date_start' : event.get('sales',{}).get('public',{}).get('startDateTime'),
            'prices': event.get('priceRanges'),
            'promoter': event.get('promoter',{}).get('name'),
            'genre' : event.get('classifications'),
            'place' : event.get('place')
         })
    tmDF = pd.DataFrame(event_info)
    return tmDF

### Pull the data and create the data frame
For each city/market, get the number of pages of data, pull all data, and then concatanate the dataframes

In [6]:
fv_df_list = []

#Create dictionaries of dma/mkt numbers, and number of pages of data
dma_dict = dict()
mkt_dict = dict()
#Get number of pages for face value dma items
for dma in dma_list:
    dma_dict[dma] = get_number_of_TM_pages(dma=dma)
    
#Get DMA data
for dma in dma_dict.keys():
    for page in range(1,dma_dict[dma]+1):
        fv_df_list.append(getTicketMasterData(dma=dma,page=page))
    
#Get number of pages for face value market items
for mkt in mkt_list:
    mkt_dict[mkt] = get_number_of_TM_pages(market=mkt)
    
#Get market data
for mkt in mkt_dict.keys():
    for page in range(1,mkt_dict[mkt]+1):
        fv_df_list.append(getTicketMasterData(market=mkt,page=page))
        
#Combine all dataframes for full data on events and face_value prices
fv_df = pd.concat(fv_df_list,axis=0)
fv_df.head()

,artist,description,genre,id,more_info,name,place,presale_date_end,presale_date_start,prices,promoter,sale_date_start,span_multiple_days,start_date,timezone,venue,venue_city,venue_state
0,[Kehlani],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",G5vYZfnXRo7ma,{},Kehlani,None,[2017-10-20T06:59:00Z],[2017-10-19T17:00:00Z],"[{'type': 'standard', 'currency': 'USD', 'min'...",ANOTHER PLANET,2017-10-20T17:00:00Z,False,2017-12-09T04:00:00Z,America/Los_Angeles,[Fox Theater - Oakland],[Oakland],[California]
1,[Phoenix],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1AvjZfVGkUn5erN,{},Phoenix,None,"[2017-10-20T06:59:00Z, 2017-10-20T06:59:00Z]","[2017-10-19T17:00:00Z, 2017-10-19T17:00:00Z]","[{'type': 'standard', 'currency': 'USD', 'min'...",ANOTHER PLANET,2017-10-20T17:00:00Z,False,2017-12-09T04:00:00Z,America/Los_Angeles,[Grand Sierra Resort and Casino],[Reno],[Nevada]
2,[Pixies],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",G5vYZfSUhyft3,{},Pixies,None,[],[],"[{'type': 'standard', 'currency': 'USD', 'min'...",PROMOTED BY VENUE,2017-10-04T19:00:00Z,False,2017-12-09T04:00:00Z,America/Los_Angeles,[Uptown Theatre Napa],[Napa],[California]
3,[Tyler Farr],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1AvjZfSGkRWZ0iG,{},Tyler Farr,None,[],[],"[{'type': 'standard', 'currency': 'USD', 'min'...",PROMOTED BY VENUE,2017-09-30T17:00:00Z,False,2017-12-09T04:00:00Z,America/Los_Angeles,[Silver Legacy Casino],[Reno],[Nevada]
4,[],{},"[{'primary': True, 'segment': {'id': 'KZFzniwn...",1Ae0ZfUGkHgbwaa,{},"Strike Iron, Baby and the Luvies",None,[],[],None,None,2017-11-15T21:45:01Z,False,2017-12-09T05:00:00Z,America/Los_Angeles,[Brick and Mortar Music Hall],[San Francisco],[California]


In [8]:
fv_df.shape

(4774, 18)